In [2]:
!pip install gradio
!pip install faiss-cpu
!pip install requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 89.5 MB/s eta 0:00:00


In [1]:
import random
import pandas as pd

In [2]:
import os
os.environ['WANDB_DISABLED'] = 'true'

Загружаем и инициализируем обученную retrieval-модель

In [ ]:
!unzip /content/e5-custom-trained_multilang.zip

In [4]:
from sentence_transformers import SentenceTransformer

In [5]:
# 3. Инициализация модели
model = SentenceTransformer("/content/content/e5-custom-trained")

##Создание векторной БД для семантического поиска
Создаем векторное хранилище документов с использованием FAISS, которое обеспечивает эффективный поиск текстовых фрагментов по семантической близости.

Объединяем тренировочную и валидационную выборки, извлекаем из них текстовые фрагменты и добавляем префикс `passage: `, чтоб E5 лучше понимала контекст.

После этого вычисляем эмбеддинги для обработанных текстов. Это нужно для вычисления близости эмбеддингов запроса пользователя и имеющимися документами в БД.

И инициализируем FAISS-индекс.

In [ ]:
import faiss

train_data_df = pd.read_csv("/content/train_data.csv")
val_data_df = pd.read_csv("/content/val_data.csv")
documents_df = pd.concat([train_data_df, val_data_df], ignore_index=True)
documents = documents_df["positive"].tolist()

prefixed_docs = [f"passage: {doc}" for doc in documents]
embeddings = model.encode(prefixed_docs, normalize_embeddings=True)
# Создание FAISS индекса
index = faiss.IndexFlatIP(embeddings.shape[1])
index.add(embeddings)
faiss.write_index(index, "e5_faiss_index.bin")

Функция `semantic_search` выполняет семантический поиск релевантных документов в векторной базе по пользовательскому запросу.

In [11]:
# Функция поиска
def semantic_search(query, top_k=12):
    prefixed_query = f"query: {query}"
    query_embedding = model.encode([prefixed_query], normalize_embeddings=True)
    distances, indices = index.search(query_embedding, top_k)

    # Убираем дубликаты из-за того, что на один абзац формируется 3 ответа
    seen = set()
    unique_results = []
    for idx, dist in zip(indices[0], distances[0]):
        doc = documents[idx]
        if doc not in seen:
            seen.add(doc)
            unique_results.append((doc, float(dist)))

    return unique_results

##Отправка запроса к LLM
Ретривер готов и теперь осталось отправить запрос к LLM, для генерации ответа по найденным релевантным документам через платформу OpenRouter.

###Архитектура RAG:
На переданнный вопрос ретривер находит релевантные документы, передает их вместе с промптом в LLM, которая генерирует финальный ответ.

Задача LLM - сгенерировать ответ на заданный вопрос, используя информацию, найденную ретривером.

In [12]:
import requests
from openai import OpenAI

In [ ]:
from google.colab import userdata

In [14]:
client7 = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=userdata.get('API_KEY7')
  )

In [15]:
def generate_answer(question):
    # Ищем релевантные абзацы
    search_results = semantic_search(question, top_k=12)
    if not search_results:
        return "Не удалось найти подходящую информацию"

    best_paragraph, score = search_results[0]
    best_paragraph1, score1 = search_results[1]
    best_paragraph2, score2 = search_results[2]


    # Промпт
    prompt = f"""
    Задача:
    Найди в предоставленных данных ответ на вопрос пользователя. Затем сформулируй ответ на вопрос пользователя.
    Ответ должен звучать как экспертное утверждение, без упоминания источников, данных или контекста. Предоставь максимум информации используя имеющиеся данные.
    Информации может быть мало, но даже в таком случае нужно сформулировать ответ на вопрос пользователя.

    Вопрос пользователя: {question}
    Данные для ответа:
    1. {best_paragraph}
    2. {best_paragraph1}
    3. {best_paragraph2}

    """

    # Запрос к LLM
    completion = client7.chat.completions.create(
      extra_body={},
      model="meta-llama/llama-4-maverick:free",
      messages=[
        {
          "role": "user",
          "content": [
            {
              "type": "text",
              "text": prompt
            }
          ]
        }
      ]
    )

    # Обработка ответа
    if completion and completion.choices:
        content = completion.choices[0].message.content
        return f"""
        Ответ: {content}\n\n

        """
    else:
        print("Ошибка: LLM не вернул ответ")


Пример работы ретривера:

In [24]:
# Пример использования
results = semantic_search("Какие мероприятия в политехе?")
for doc, score in results:
    print(f"Score: {score:.4f} | {doc}")

Score: 0.4839 | самое теплое мероприятие, которое как нельзя кстати проходит

- «Леденец» в холодную зимнюю пору! Настоящее катание на коньках, конкурсы, призы уверены, ты уже ждешь!
Score: 0.4328 | И это далеко не всё! В течение всего курса каждый сможет найти себе мероприятие по душе!
Score: 0.4278 | Во время летних каникул ребят ждет отдых в спортивно-оздоровительном лагере «Ждановец». Руководство политеха активно взаимодействует со студенческим активом и студенческими объединениями.
Score: 0.4269 | Тогда мы ждем именно тебя! У нас своя репетиционная точка, мы располагаем большим количеством инструментов и выступаем на главных мероприятиях Политеха и Ждановца, а также за их пределами.


##Создание графического интерфейса
Для обеспечения удобного взаимодействия пользователей с RAG-системой реализован веб-интерфейс с использованием библиотеки `gradio`.

In [17]:
import gradio as gr

In [25]:
iface = gr.Interface(
    fn=generate_answer,
    inputs=gr.Textbox(label="Ваш вопрос", placeholder="Введите ваш вопрос здесь"),
    outputs=gr.Textbox(label="Результаты поиска", lines=10),
    title="Генерация ответов",
    description="Введите вопрос и получите релевантный ответ"
)
iface.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://44a14ea52017d49a57.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
